### (1)Check the version of Pytorch and Cuda

In [10]:
import torch
print(torch.__version__)

2.2.1+cu121


In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


### (2)Install Unsloth (and Pytorch, if needed)

In [ ]:
%%capture
#!pip uninstall torch torchvision -y
#!pip install torch==2.2.1+cu121 torchvision==0.17.1+cu121 -f https://download.pytorch.org/whl/torch_stable.html
#!pip install torch==2.3.0+cu121 torchvision==0.18.0+cu121 -f https://download.pytorch.org/whl/torch_stable.html
import torch
major_version, minor_version = torch.cuda.get_device_capability()
# Due to Colab has torch2.2.1, we need to install it seperately
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    # New GPU，Ex: Ampere、Hopper GPU（RTX 30xx、RTX 40xx、A100、H100、L40）。
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    # Old GPU（V100、Tesla T4、RTX 20xx）
    !pip install --no-deps trl peft accelerate bitsandbytes
    !pip install xformers==0.0.25  #v0.026 incompatible
pass

In [ ]:
# Check nvidia status
#!nvidia-smi


In [ ]:
#check GPU
import torch
print(torch.cuda.is_available())  # 如果 PyTorch 正在使用 GPU，則輸出為 True，否則為 False。

import tensorflow as tf
print(tf.test.gpu_device_name())  # 標準輸出為 '/device:GPU:0'


In [ ]:
# Check whether installed the correct version
#!pip show torch
#!pip show torchvision

In [ ]:
# Check env
!python -m torch.utils.collect_env

In [ ]:
# Install package if it is missed
#!pip install torch-functorch
#!pip install functorch
#!pip install dynamo

### (3)Load Model

In [ ]:
#2 Load Model
#import torch
#import torch._dynamo
#import dynamo
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

### (4)Test before fine-tuning

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}
### Input:
{}
### Response:
{}"""

FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "請用中文回答", # instruction
        "海綿寶寶的書法是不是叫做海綿體？", # input
        "", # output
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

### (5)Prepare dataset for fine-tuning

In [ ]:
#4准备微调数据集
EOS_TOKEN = tokenizer.eos_token # 必须添加 EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # 必须添加EOS_TOKEN，否则无限生成
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("HeineChu/ruozhiba", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Generating train split:   0%|          | 0/1496 [00:00<?, ? examples/s]

Map:   0%|          | 0/1496 [00:00<?, ? examples/s]

### (6)Set training parameters

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

model = FastLanguageModel.get_peft_model(
    model,
    r = 16, #  suggest 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth", # checkpoint for the length of upper,lower part's paragraph
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # faster 5x times for short paragraph
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,  # fine-tune steps
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Unsloth 2024.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.
/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=2):   0%|          | 0/1496 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


### (7)Start training

In [ ]:
trainer_stats = trainer.train()

### (8)Test model(after fine-tuning)

In [ ]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "只用中文回答問題", # instruction
        "天堂離地球有多遠啊，爸媽說我奶奶去了那，怎麼好幾年了還沒回來", # input
        "", # output
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

### (9)Store LoRA model

In [ ]:
model.save_pretrained("lora_model") # Local saving
# model.push_to_hub("your_name/lora_model", token = "...") # Keep it to hugging face (needs token)

### (10)Merge Model and quantize it into a 4-bit gguf and save it

In [ ]:
model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
#model.save_pretrained_merged("outputs", tokenizer, save_method = "merged_16bit",) #Merge and stor as 16bits hf
#model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "") #Merge to 4bits gguf，upload to hugging face(needs token)

### (11)Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### (12)Copy Model to Google Drive

In [ ]:
import shutil
source_file = '/content/model-unsloth.Q4_K_M.gguf'
destination_dir = '/content/drive/MyDrive/Llama3'
destination_file = f'{destination_dir}/model-unsloth.Q4_K_M.gguf'
shutil.copy(source_file, destination_file)